# 1125

## Hello C++

In [13]:
%%bash
cat > hello.cpp << CPPCODE

#include <iostream>
using namespace std;
int main() {
    cout << "hello" << endl;
}

CPPCODE
g++-14 -std=c++26 -Wall -Wextra -Wpedantic -Wshadow -Wconversion hello.cpp -o h && ./h # ~ && ~ means run the compiled program if compilation is successful

hello


## Hello Python

In [14]:
%%bash
cat > hello.py << PYCODE    

print("Hello World")

PYCODE

In [15]:
!python3 hello.py

Hello World


## Hello Cython

In [16]:
%%bash
cat > setup.py << PYCODE

from setuptools import setup                # Import setup from setuptools
from Cython.Build import cythonize          # Import cythonize from Cython.Build
setup(                                      # Call setup function
    ext_modules = cythonize("hello.py")     # Specify the Cython file to be compiled (cythonize function is used to compile Cython files)
)

PYCODE

In [17]:
!python setup.py build_ext --inplace        # Compile the Cython code to a shared object file

Compiling hello.py because it changed.
[1/1] Cythonizing hello.py
running build_ext
building 'hello' extension
gcc -pthread -B /home/maxlin0116/miniforge3/envs/sage/compiler_compat -fno-strict-overflow -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /home/maxlin0116/miniforge3/envs/sage/include -fPIC -O2 -isystem /home/maxlin0116/miniforge3/envs/sage/include -fPIC -I/home/maxlin0116/miniforge3/envs/sage/include/python3.12 -c hello.c -o build/temp.linux-x86_64-cpython-312/hello.o
gcc -pthread -B /home/maxlin0116/miniforge3/envs/sage/compiler_compat -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/home/maxlin0116/miniforge3/envs/sage/lib -Wl,-rpath-link,/home/maxlin0116/miniforge3/envs/sage/lib -L/home/maxlin0116/miniforge3/envs/sage/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/home/maxlin0116/miniforge3/envs/sage/lib -Wl,-rpath-link,/home/maxlin0116/miniforge3/envs/sage/lib -L/home/maxlin0116/miniforge3/envs/sage/lib build/temp.linux-x86_64-cpython-312/hello.o -o build/lib.linux-x86_

In [18]:
import hello                                # Import the compiled Cython module

## Hello Cell Magic

In [19]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [20]:
%%cython
cpdef int add(int a, int b): # works for both Python and Cython
    return a + b + 1

In [21]:
add(2, 3)

6

## Hello Fibonacci

In [22]:
def fib(n):
    """Print the Fibonacci series up to n."""
    if n <= 1: return 1
    return fib(n-1) + fib(n-2)

In [23]:
fib(1), fib(2), fib(3), fib(4), fib(5)

(1, 2, 3, 5, 8)

In [24]:
%%timeit

fib(10)

21.8 μs ± 667 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
def fib1(n):
    """Print the Fibonacci series up to n."""
    if n <= 1: return (0, )
    return (fib(n-1) , fib(n-2))

In [26]:
fib1(1), fib1(2), fib1(3), fib1(4), fib1(5)

((0,), (1, 1), (2, 1), (3, 2), (5, 3))

In [27]:
%%timeit

fib1(10)

22 μs ± 504 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
%%cython
cpdef int fibo(int n):
    if n <= 1: return 1
    return fibo(n-1) + fibo(n-2)

In [29]:
fibo(1), fibo(2), fibo(3), fibo(4), fibo(5)

(1, 2, 3, 5, 8)

In [30]:
%%timeit

fibo(10)

271 ns ± 7.51 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Cython Include C

In [36]:
%%bash

cat > fibon.hpp << HPPCODE
    #pragma once
    long long fibon(int n);
HPPCODE

cat > fibon.cpp << CPPCODE
    long long fibon(int n) {
        if (n <= 1) return 1;
        return fibon(n - 1) + fibon(n - 2);
    }
CPPCODE
g++-14 -std=c++26 -O3 -fPIC -shared fibon.cpp -o libfibon.so #-O3 for optimization, -fPIC for position independent code, -shared to create a shared object file

In [32]:
%%bash
ls libfibon.so fibon.hpp # shared object file means it can be linked to other programs at runtime

fibon.hpp
libfibon.so


In [33]:
%%bash
ls

1125.ipynb
build
fibon.cpp
fibon.hpp
h
hello.c
hello.cpp
hello.cpython-312-x86_64-linux-gnu.so
hello.py
libfibon.so
setup.py


In [34]:
%%cython --cplus -I. -L. -lfibon --link-args=-Wl,-rpath,.

# C++ mode, include + lib dirs = current directory
# --link-args passes extra_link_args to the linker
# lib~.so is the shared file of ~.cpp

cdef extern from "fibon.hpp":       # Declaration of the C++ function(extern means the function is defined elsewhere)
    long long fibon(int n)

def fibon_py(int n):
    """
    Python wrapper around C++ fibon(int n).
    """
    return fibon(n)

In [35]:
%%timeit

fibon_py(10)

117 ns ± 4.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
